### A code which creates the Wigner interactive function in 2D and 3D with different squeezing parameters.

In [26]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

First testing out the interactive plots

In [2]:
def model(x,A):
    return np.sin(x)*A

def interactive_plot(A):
    x = np.linspace(0,10,100)
    y = model(x,A)
    plt.plot(x,y)
    plt.grid(True)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'{A}sin(x)')
    return

interact(interactive_plot, A=widgets.FloatSlider(value=1, min=-10, max=10, step=0.1))


interactive(children=(FloatSlider(value=1.0, description='A', max=10.0, min=-10.0), Output()), _dom_classes=('…

<function __main__.interactive_plot(A)>

Defining the Wigner function

In [3]:
# Create functions for calculating the Wigner function of a general Gaussian state.

def WignerFunction(x,p,x_displacement,y_displacement,sigma,n=1): 

    r_vec = np.array([x, p])
    
    x_mean = np.mean(x) 
    p_mean = np.mean(p) 

    r_vec_mean = np.array([x_displacement,y_displacement])
    
    delta = r_vec - r_vec_mean

    

    Wigner = ( 1 / ( (2*np.pi)**n * np.sqrt(np.linalg.det(sigma)) ) ) * np.exp( -0.5 * (delta).T @ np.linalg.inv(sigma) @ (delta) )
    return Wigner

In [ ]:
def Wigner_Plot(sigma,x_displacement,y_displacement):
    x = np.linspace(-5,5,200)
    p = np.linspace(-5,5,200)

    X,P = np.meshgrid(x,p)

    W = np.zeros_like(X)

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            W[i, j] = WignerFunction(X[i, j], P[i, j],x_displacement,y_displacement,sigma)

    return W, X, P


Creating a function which changes sigma depending on the squeezing factor

In [45]:
sigma_vacuum = 0.5*np.eye(2)
x_vacuum,p_vacuum = 0,0

r = 1

wigner_sqz_x = 0.5*np.array([
    [np.exp(-2*r),0],
    [0,np.exp(2*r)]
])

wigner_sqz_p = 0.5*np.array([
    [np.exp(2*r),0],
    [0,np.exp(-2*r)]
])

def sigma_func(r):
    sigma_sqz = 0.5*np.array([ [np.exp(2*r),0], [0,np.exp(-2*r)]])
    
    return sigma_sqz

def sigma_func_theta(r, theta):
    S_theta = np.array ([[np.cos(theta), np.sin(theta)],
                         [np.sin(theta), -np.cos(theta)]])
    sigma_sqz_theta = 0.5 * np.cosh(2*r)*np.eye(2) - 0.5 * np.sinh(2*r)*S_theta
    
    return sigma_sqz_theta

#wigner_vacuum, X, P  = Wigner_Plot(sigma_vacuum,x_vacuum,p_vacuum)
#wigner_sqz_x, X, P = Wigner_Plot(wigner_sqz_x,0,0)
wigner_sqz_p, X, P = Wigner_Plot(sigma_func(0.1),0,0)


In [48]:
def interactive_Wigner_Plot(r, x0, p0, theta):
    
    W = Wigner_Plot(sigma_func_theta(r, theta),x0,p0)

    plt.figure(figsize=(8,6))
    plt.contourf(X, P, W[0], levels=50, cmap='magma')
    plt.colorbar(label='Wigner function')
    plt.xlabel('x')
    plt.ylabel('p')
    plt.title('2D Wigner function plot')
    plt.show()
    return

In [51]:
interact(interactive_Wigner_Plot, 
         r=widgets.FloatSlider(value=0, min=-3, max=3, step=0.1),
         x0=widgets.FloatSlider(value=0.0, min=-4.0, max=4.0, step=0.1, description='x'),
         p0=widgets.FloatSlider(value=0.0, min=-4.0, max=4.0, step=0.1, description='p'),
         theta=widgets.FloatSlider(value=0.0, min=-np.pi, max=np.pi, step=0.1, description='theta'))

interactive(children=(FloatSlider(value=0.0, description='r', max=3.0, min=-3.0), FloatSlider(value=0.0, descr…

<function __main__.interactive_Wigner_Plot(r, x0, p0, theta)>

In [ ]:
def interactive_Wigner_Plot_3D(r, x0, p0, theta):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    W,_,_ = Wigner_Plot(sigma_func_theta(r, theta), x0, p0)

    ax.plot_surface(X, P, W, cmap='magma', edgecolor='none')

    ax.set_xlabel('x')
    ax.set_ylabel('p')
    ax.set_zlabel('W(x, p)')
    ax.set_title('3D Wigner Function')

    plt.tight_layout()
    plt.show()

    return

interact(interactive_Wigner_Plot_3D, 
         r=widgets.FloatSlider(value=0, min=-1, max=1, step=0.1),
         x0=widgets.FloatSlider(value=0.0, min=-4.0, max=4.0, step=0.1, description='x₀'),
         p0=widgets.FloatSlider(value=0.0, min=-4.0, max=4.0, step=0.1, description='p₀'),
         theta=widgets.FloatSlider(value=0.0, min=-2*np.pi, max=2*np.pi, step=0.1, description='theta'))

interactive(children=(FloatSlider(value=0.0, description='r', max=1.0, min=-1.0), FloatSlider(value=0.0, descr…

<function __main__.interactive_Wigner_Plot_3D(r, x0, p0, theta)>